In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
from pathlib import Path

from matplotlib import pyplot as plt
# from tqdm import tqdm
from tqdm.notebook import tqdm
from pathlib import Path

### Load a classifier

In [2]:
from collab_splats.utils.grouping import GroupingClassifier, GroupingConfig

# Path to the config for a trained model
load_config = '/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-25_074037/config.yml'
load_config = Path(load_config)

grouping_config = GroupingConfig(segmentation_backend='mobilesamv2', segmentation_strategy='object', front_percentage=0.2, iou_threshold=0.1, num_patches=32)
grouping_classifier = GroupingClassifier(load_config=load_config, config=grouping_config)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Taichi] version 1.7.4, llvm 15.0.4, commit b4b956fd, linux, python 3.10.18


[I 09/22/25 21:30:03.398 51632] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


Memory bank loaded from /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/grouping/memory_bank.pkl with 729 masks


Process ForkProcess-22:
Process ForkProcess-9:
Process ForkProcess-32:
Process ForkProcess-29:
Process ForkProcess-20:
Process ForkProcess-18:
Process ForkProcess-24:
Process ForkProcess-4:
Process ForkProcess-19:
Process ForkProcess-31:
Process ForkProcess-17:
Process ForkProcess-5:
Process ForkProcess-1:
Process ForkProcess-26:
Process ForkProcess-23:
Process ForkProcess-6:
Process ForkProcess-21:
Process ForkProcess-12:
Process ForkProcess-8:
Process ForkProcess-11:
Process ForkProcess-28:
Process ForkProcess-2:
Process ForkProcess-27:
Process ForkProcess-10:
Process ForkProcess-7:
Process ForkProcess-3:
Traceback (most recent call last):
Process ForkProcess-30:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (

### Associate masks

In [15]:
grouping_classifier.create_masks()

Loading train ordered data...


Creating masks [train]: 100%|██████████| 441/441 [00:02<00:00, 165.34it/s]

No object detected.
No objects found in frame_00232.png, creating empty mask
No object detected.
No objects found in frame_00235.png, creating empty mask


In [16]:
grouping_classifier.associate()

Loading train ordered data...


Processing frames:  52%|█████▏    | 228/441 [00:02<00:01, 124.67it/s]

No objects found in frame_00232.png
No objects found in frame_00235.png


Processing frames: 100%|██████████| 441/441 [00:04<00:00, 93.13it/s] 


### Try pytorch lightning datamodule

In [5]:
from collab_splats.utils.grouping import GroupingDataModule 

datamodule = GroupingDataModule(
    datamanager=grouping_classifier.pipeline.datamanager,
    mask_dir=grouping_classifier.associated_mask_dir,
    device="cuda",
    train_num_workers=0,
    val_num_workers=0,
)

Setup classifier for training

In [18]:
grouping_classifier.setup()

Train identity embeddings to lift objects from 2d to 3d

In [19]:
grouping_classifier.lift_segmentation()

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                | Params | Mode 
-----------------------------------------------------------
0 | pipeline   | VanillaPipeline     | 465 M  | eval 
1 | model      | RadegsFeaturesModel | 465 M  | eval 
2 | params     | ParameterDict       | 16.8 M | train
3 | classifier | Conv2d              | 10.2 K | train
4 | loss_fn    | CrossEntropyLoss    | 0      | train
--------------------------------------------------------

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=10000` reached.


(GroupingClassifier(
   (pipeline): VanillaPipeline(
     (_model): RadegsFeaturesModel(
       (gauss_params): ParameterDict(
           (features_dc): Parameter containing: [torch.FloatTensor of size 1295257x3]
           (features_rest): Parameter containing: [torch.FloatTensor of size 1295257x0x3]
           (means): Parameter containing: [torch.FloatTensor of size 1295257x3]
           (opacities): Parameter containing: [torch.FloatTensor of size 1295257x1]
           (quats): Parameter containing: [torch.FloatTensor of size 1295257x4]
           (scales): Parameter containing: [torch.FloatTensor of size 1295257x3]
           (distill_features): Parameter containing: [torch.FloatTensor of size 1295257x13]
       )
       (camera_optimizer): CameraOptimizer()
       (psnr): PeakSignalNoiseRatio()
       (ssim): SSIM()
       (lpips): LearnedPerceptualImagePatchSimilarity(
         (net): _NoTrainLpips(
           (scaling_layer): ScalingLayer()
           (net): Alexnet(
          

### Try to map onto the mesh?

In [38]:
import pickle
import open3d as o3d
from collab_splats.utils.mesh import features2vertex


mesh_dir = grouping_classifier.output_dir.parent / 'mesh'

mesh_path = mesh_dir / 'mesh.ply'
transforms_path = mesh_dir / 'transforms.pkl'

with open(transforms_path, 'rb') as f:
    transforms = pickle.load(f)

mesh = o3d.io.read_triangle_mesh(mesh_path)


In [31]:
mesh

TriangleMesh with 592404 points and 1171436 triangles.

In [34]:
means = grouping_classifier.model.means.clone()

means = means @ transforms["mesh_transform"][:3, :3].T + transforms["mesh_transform"][:3, 3]

In [68]:
classes = grouping_classifier.per_gaussian_forward(grouping_classifier.identities)
classes = classes.argmax(-1).unsqueeze(-1)

In [70]:
features = features2vertex(
    mesh_vertices=mesh.vertices,
    points=means,
    features=classes,
    categorical=True
)

Mapping categorical features: 100%|██████████| 5/5 [00:17<00:00,  3.48s/it]


In [96]:
# Create RGB colors for each unique class
unique_classes = torch.unique(features)
n_classes = len(unique_classes)

# Generate distinct colors using HSV colorspace for better visual separation
import matplotlib.pyplot as plt
cmap = plt.get_cmap('tab10')  # or 'viridis', 'plasma', etc.

# Create color mapping
class_to_rgb = {}
for i, class_id in enumerate(unique_classes):
    color = cmap(i / max(1, n_classes - 1))  # Normalize to [0,1]
    class_to_rgb[class_id.item()] = torch.tensor(color[:3], dtype=torch.float32)  # RGB only

# Map classes to RGB colors
rgb_colors = torch.zeros(features.shape[0], 3, dtype=torch.float32)
for i, class_id in enumerate(features.squeeze()):
    rgb_colors[i] = class_to_rgb[class_id.item()]

rgb_colors

tensor([[0.1216, 0.4667, 0.7059],
        [0.1216, 0.4667, 0.7059],
        [0.1216, 0.4667, 0.7059],
        ...,
        [0.5804, 0.4039, 0.7412],
        [0.5804, 0.4039, 0.7412],
        [0.1216, 0.4667, 0.7059]])

In [97]:
import pyvista as pv

mesh = pv.read(mesh_path.as_posix())

print(mesh.n_points)
print(mesh.n_cells)
print(mesh.bounds)

mesh.plot(
    scalars=rgb_colors, 
    rgb=True
)

592404
1171436
BoundsTuple(x_min=-1.149007797241211, x_max=0.964453399181366, y_min=-0.3168279230594635, y_max=1.4859158992767334, z_min=-0.9972854256629944, z_max=-0.17026546597480774)


Widget(value='<iframe src="http://localhost:42685/index.html?ui=P_0x7e04d03150f0_12&reconnect=auto" class="pyv…

In [75]:
mesh.vertex_colors = o3d.utility.Vector3dVector(features.numpy())

RuntimeError: Unable to cast Python instance of type <class 'numpy.int64'> to C++ type '?' (#define PYBIND11_DETAILED_ERROR_MESSAGES or compile in debug mode for details)

In [23]:
for x in datamodule.train_dataloader():
    break

outs = grouping_classifier(x[0])

RuntimeError: t == DeviceType::CUDA INTERNAL ASSERT FAILED at "/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/torch/include/c10/cuda/impl/CUDAGuardImpl.h":25, please report a bug to PyTorch. 